In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
import cvxpy as cp
import numpy as np

In [ ]:
from solara.constants import PROJECT_PATH
import solara.environment.photovoltaics
import solara.environment.loads
import solara.environment.grids
pv_data_path = PROJECT_PATH + "/data/solar_trace_data/PV_5796.txt"
load_data_path = PROJECT_PATH + "/data/solar_trace_data/load_5796.txt"

pv_model = solara.environment.photovoltaics.DataPV(data_path=pv_data_path)
load_model = solara.environment.loads.DataLoad(data_path=load_data_path)
grid_model = solara.environment.grids.PeakGrid()

pv_model.fix_start(1)
load_model.fix_start(1)

In [ ]:
start = np.random.randint(len(pv_model.data) // 24) * 24

pv_model.reset()
load_model.reset()
print("Starting point: ", pv_model.start)
for i in range(24):
    load = load_model.get_next_load()
    pv_gen = pv_model.get_next_generation()
    net_load = np.maximum(0,load-pv_gen)
    price = grid_model.draw_power(net_load)
    print("Load: {:.3f}, PV generation: {:.3f}, Net load: {:.3f}, Price: {:.3f}".format(load, pv_gen, net_load, price))

In [ ]:
import solara.environment.battery_control
import solara.environment.batteries

battery_model = solara.environment.batteries.LithiumIonBattery(20, "NMC", 1/10.0)
pv_model = solara.environment.photovoltaics.DataPV(data_path=pv_data_path)
load_model = solara.environment.loads.DataLoad(data_path=load_data_path)
grid_model = solara.environment.grids.PeakGrid()

start = 12

pv_model.fix_start(start=start)
load_model.fix_start(start=start)

env = solara.environment.battery_control.BatteryControlEnv(battery=battery_model,
        pv_system = pv_model,
        grid = grid_model,
        load = load_model,)

In [ ]:
done = False
env.reset()
obs_list = []
for i in range(26):
    if not done:
        obs, _, done, _ = env.step(np.array([0]))
        obs_list.append(obs)
    else:
        print("Episode done.")

In [ ]:
import pandas as pd

df = pd.DataFrame(data=obs_list)
df.columns = ["load",
            "pv_generation",
            "battery__energy_content",
            "time_step",
            "sum_load",
            "sum_pv_gen",]

y_names = ["load",
            "pv_generation",
            "battery__energy_content",]
            #"sum_load",
            #"sum_pv_gen",]
        
df.plot(x="time_step", y=y_names, secondary_y=True)